# PA 7.1

## Loading in All of the Notes Chunks

In [18]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


In [19]:
ames = pd.read_csv("/Users/stolieerickson/Documents/MSBA/Fall/GSB544-Computing/InClass/Wk7/AmesHousing.csv")
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,...,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,...,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,...,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,...,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,...,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,...,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


## Practice Activity
Consider four possible models for predicting house prices:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [20]:
#predictors and target
X = ames[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']]
y = ames['SalePrice']

#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#preprocessor
numeric_features = ['Gr Liv Area', 'TotRms AbvGrd']
categorical_features = ['Bldg Type']

basic_preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(drop='first'), categorical_features)
], remainder='drop')

#pipelines for each model
#model 1: size and number of rooms
model_1 = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('num', StandardScaler(), numeric_features)
    ], remainder='drop')),
    ('model', LinearRegression())
])

#model 2: size, number of rooms, and building type
model_2 = Pipeline([
    ('preprocessor', basic_preprocessor),
    ('model', LinearRegression())
])

#model 3: size, building type, and their interaction
model_3 = Pipeline([
    ('preprocessor', basic_preprocessor),
    ('interaction', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
    ('model', LinearRegression())
])

#model 4: polynomial 5 on size and rooms, and building type
model_4 = Pipeline([
    ('preprocessor', basic_preprocessor),
    ('poly_features', PolynomialFeatures(degree=5, include_bias=False)),
    ('model', LinearRegression())
])

#dictionary for iteration
pipelines = {
    "model_1": model_1,
    "model_2": model_2,
    "model_3": model_3,
    "model_4": model_4
}

#fit and evaluate
rmse_scores = {}
for name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    rmse_scores[name] = mean_squared_error(y_test, y_pred, squared=False)

#best model
best_model = min(rmse_scores, key=rmse_scores.get)
print(f"RMSE Scores: {rmse_scores}")
print(f"Best Model: {best_model}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use 

RMSE Scores: {'model_1': 51020.35394090045, 'model_2': 49737.74135592328, 'model_3': 49300.87902888169, 'model_4': 62723.925214968505}
Best Model: model_3
RMSE Scores: {'model_1': 51020.35394090045, 'model_2': 49737.74135592328, 'model_3': 49300.87902888169, 'model_4': 62723.925214968505}
Best Model: model_3


## Practice Activity
Once again consider four modeling options for house price:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?
The preferred model using cross val score is model 3, which agrees with my conclusion from earlier.

In [21]:
#use previous pipeline to evaluate
cv_rmse_scores = {}
for name, pipeline in pipelines.items():
    #cross_val_score
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    cv_rmse_scores[name] = -np.mean(scores)  # Convert to positive RMSE

#print scores
print(f"Cross-validated RMSE Scores: {cv_rmse_scores}")

#best model
best_model = min(cv_rmse_scores, key=cv_rmse_scores.get)
print(f"Best Model by Cross-Validated RMSE: {best_model}")

Cross-validated RMSE Scores: {'model_1': 56663.06248423473, 'model_2': 54753.33632743881, 'model_3': 53687.70850488586, 'model_4': 204902.86772070933}
Best Model by Cross-Validated RMSE: model_3
Cross-validated RMSE Scores: {'model_1': 56663.06248423473, 'model_2': 54753.33632743881, 'model_3': 53687.70850488586, 'model_4': 204902.86772070933}
Best Model by Cross-Validated RMSE: model_3


## Practice Activity
Consider one hundred modeling options for house price:

House size, trying degrees 1 through 10
Number of rooms, trying degrees 1 through 10
Building Type

Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

The degree 2 model performed the best.

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

Long computation time, and potential to overfit the data. I would investigate the relationship for each variable with the response and see if the true relationship should be captured by a polynomial. Consider Lasso or Ridge regression for tuning. 

In [22]:
#predictors and target
X = ames[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']]
y = ames['SalePrice']

#preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), ['Gr Liv Area', 'TotRms AbvGrd']),
    ('cat', OneHotEncoder(drop='first'), ['Bldg Type'])
])

#pipeling for each model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly_features', PolynomialFeatures()),
    ('model', LinearRegression())
])

#grid with degrees 1-10
param_grid = {
    'poly_features__degree': range(1, 11), 
}

#gridsearch (5 fold)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_search.fit(X, y)

#score and find best model
best_degree = grid_search.best_params_['poly_features__degree']
best_rmse = -grid_search.best_score_

print(f"Best Degree: {best_degree}")
print(f"Best Cross-Validated RMSE: {best_rmse}")

Best Degree: 2
Best Cross-Validated RMSE: 53462.10347529638
Best Degree: 2
Best Cross-Validated RMSE: 53462.10347529638


In [23]:
#quiz question
#pull best model
best_model = grid_search.best_estimator_

#predict on training
y_pred = best_model.predict(X)

#pull r squared
r_squared = r2_score(y, y_pred)

#print
print(f"R-squared of the Best Model: {r_squared}")

R-squared of the Best Model: 0.5741476945191711
R-squared of the Best Model: 0.5741476945191711
